In [21]:
import datetime
import pandas as pd
import numpy as np
from fbprophet import Prophet
import glob
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import cmath

df_final = pd.DataFrame()
df_final = df_final.append(pd.read_csv("csv_files/211011117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211021117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211031117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211041117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211051117.17N.csv"))

df_expect = pd.read_csv("csv_files/211061117.17N.csv")

#df_final = pd.DataFrame()
#for file in glob.glob("csv_files/*.csv"):
#  print(file)
#  df = pd.read_csv(file)
#  df_final = df_final.append(df, ignore_index=True)

df_final['epoch_time']=pd.to_datetime(df_final['epoch_time'], format='%Y-%m-%d %H:%M:%S')
df_final = df_final.sort_values(by=['epoch_time'])
df_prn_2 = df_final.loc[df_final['prn'] == 3].reset_index()

df_expect['epoch_time']=pd.to_datetime(df_expect['epoch_time'], format='%Y-%m-%d %H:%M:%S')
df_expect = df_expect.sort_values(by=['epoch_time'])
df_expect2 = df_expect.loc[df_expect['prn'] == 3].reset_index()

future = pd.DataFrame()
future['ds'] = df_expect2['epoch_time']


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mapr = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return 0.0 if mapr!=mapr else mapr


df_predicted = pd.DataFrame()
avg_acc = 0.0

df_predicted['epoch_time'] = df_expect2['epoch_time']


a = datetime.datetime.now().replace(microsecond=0)


for column in list(df_prn_2.columns.values):
    if column == 'prn' or column == 'epoch_time' or column=='index':
        continue
    
  	#print('column  :'+ str(column))
    if column == 'sv_clock_bias' or column == 'sv_clock_drift' or column == 'sv_clock_drift_rate' or column == 'mean_motion'  or  column == 'essentricity'  or column == 'sqrt_semi_major_axis'  or column == 'OMEGA' or column == 'inclination'  or column == 'omega' or column == 'OMEGA_dot' or column == 'inclination_rate' or column == 'codes' or column == 'gps_week' or column == 'l2_p_data_flag' or column == 'sv_accuracy' or column == 'sv_health' or column == 'tgd'  or column == 'fit_interval':
        df2 = pd.DataFrame()
        df2['ds'] = df_prn_2['epoch_time']
        df2['y'] = df_prn_2[column]

        model = Prophet()
        model.fit(df2)

        forecast=model.predict(future)
        df_predicted[column] = forecast['yhat']
        mse = mean_absolute_percentage_error(forecast['yhat'],df_expect2[column])

        avg_acc = avg_acc + mse
        #print("MSE: "+str(mse))
        del df2


    if column == 'iode' or column == 'correction_radius_sine' or  column == 'correction_latitude_cosine' or column == 't_tx' or column == 'correction_latitude_sine' or  column == 'correction_inclination_cosine' or   column == 'correction_radius_cosine' or column == 'mean_anomaly' or  column == 'correction_inclination_sine':

        df2 = pd.DataFrame()
        df2['ds'] = df_prn_2['epoch_time']
        df2['y'] = df_prn_2[column]

        model = Prophet(changepoint_prior_scale=0.5)
        model.fit(df2)

        forecast=model.predict(future)
        df_predicted[column] = forecast['yhat']
        mse = mean_absolute_percentage_error(forecast['yhat'],df_expect2[column])

        avg_acc = avg_acc + mse
        #print("MSE: "+str(mse))
        del df2

    if column == 'time_of_ephemeris' or column == 'iodc' :

        df2 = pd.DataFrame()
        df2['ds'] = df_prn_2['epoch_time']
        df2['y'] = df_prn_2[column]

        model = Prophet(changepoint_prior_scale=1.0)
        model.fit(df2)

        forecast=model.predict(future)
        df_predicted[column] = forecast['yhat']
        mse = mean_absolute_percentage_error(forecast['yhat'],df_expect2[column])

        avg_acc = avg_acc + mse
        del df2



b = datetime.datetime.now().replace(microsecond=0)
print("Time taken: ")
print(b-a)

print(df_predicted.head())

#df_expect

df_predicted.to_csv(r'predict_1_sat.csv')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.
/home/ask149/Env/gsc3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning:

invalid value encountered in true_divide

INFO:fb

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.


Time taken: 
0:00:56
           epoch_time  sv_clock_bias  sv_clock_drift  sv_clock_drift_rate  \
0 2017-11-06 14:00:00      -0.000046    5.430377e-12                  0.0   
1 2017-11-06 16:00:00      -0.000046    5.433337e-12                  0.0   
2 2017-11-06 18:00:00      -0.000046    5.437514e-12                  0.0   
3 2017-11-06 20:00:00      -0.000046    5.441690e-12                  0.0   

        iode  correction_radius_sine   mean_motion  mean_anomaly  \
0  18.124511                5.927345  5.184129e-09      1.891459   
1  20.462384                7.958005  5.196637e-09      2.942291   
2  26.700635                9.828176  5.224888e-09     -2.290780   
3  25.546606               14.321172  5.266581e-09     -1.240990   

   correction_latitude_cosine  essentricity      ...       inclination_rate  \
0                2.966674e-07      0.001102      ...          -2.120335e-10   
1                3.860445e-07      0.001103      ...          -1.829277e-10   
2              

In [22]:
import datetime
import pandas as pd
import numpy as np
from fbprophet import Prophet
import glob
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import cmath
import pickle

df_final = pd.DataFrame()
df_final = df_final.append(pd.read_csv("csv_files/211011117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211021117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211031117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211041117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211051117.17N.csv"))

df_expect = pd.read_csv("csv_files/211061117.17N.csv")

#df_final = pd.DataFrame()
#for file in glob.glob("csv_files/*.csv"):
#  print(file)
#  df = pd.read_csv(file)
#  df_final = df_final.append(df, ignore_index=True)

df_final['epoch_time']=pd.to_datetime(df_final['epoch_time'], format='%Y-%m-%d %H:%M:%S')
df_final = df_final.sort_values(by=['epoch_time'])
df_prn_2 = df_final.loc[df_final['prn'] == 3].reset_index()

df_expect['epoch_time']=pd.to_datetime(df_expect['epoch_time'], format='%Y-%m-%d %H:%M:%S')
df_expect = df_expect.sort_values(by=['epoch_time'])
df_expect2 = df_expect.loc[df_expect['prn'] == 3].reset_index()

future = pd.DataFrame()
future['ds'] = df_expect2['epoch_time']


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mapr = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return 0.0 if mapr!=mapr else mapr


df_predicted = pd.DataFrame()
avg_acc = 0.0

df_predicted['epoch_time'] = df_expect2['epoch_time']


a = datetime.datetime.now().replace(microsecond=0)
df2 = pd.DataFrame()
df2['ds'] = df_prn_2['epoch_time']
df2['y'] = df_prn_2['sv_clock_bias']

model = Prophet()
model.fit(df2)

with open('file_name', "wb") as f:
    pickle.dump(model,f)

b = datetime.datetime.now().replace(microsecond=0)
print("Time taken: ")
print(b-a)



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.


Time taken: 
0:00:00


In [23]:
model2 = Prophet()

In [24]:

with open("file_name", "rb") as f:
    model2= pickle.load(f)

In [25]:
a = datetime.datetime.now().replace(microsecond=0)
forcast = model2.predict(future)
forcast['yhat']
b = datetime.datetime.now().replace(microsecond=0)
print("Time taken: ")
print(b-a)

Time taken: 
0:00:02


In [26]:
import os
import pandas as pd
import numpy as np
from fbprophet import Prophet
import glob
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import cmath
import datetime
import multiprocessing as mp
from multiprocessing import Pool

def getdotn(cfname):
    head = '     2.10           N: GPS NAV DATA                         RINEX VERSION / TYPE\nConvert v1.7.0                          08-May-18 09:05     PGM / RUN BY / DATE \n    0.1211D-07 -0.7451D-08 -0.1192D-06  0.5960D-07          ION ALPHA           \n    0.9626D+05 -0.3277D+05 -0.1966D+06  0.1966D+06          ION BETA            \n     .279396772385D-08  .355271367880D-14   405504     1973 DELTA-UTC: A0,A1,T,W\n    18                                                      LEAP SECONDS\n                                                            END OF HEADER\n'
    row = []
    row.append(head)
    df = pd.read_csv(cfname)
    for line in open(cfname):
        row.append(line.split(','))
    index=0
    ind = 0
    stn_code = '211'
    dt = datetime.datetime.strftime(datetime.datetime.strptime(df['epoch_time'][0],'%Y-%m-%d %H:%M:%S'),'%y %m %d %H %M %S')
    re = dt.split(" ")
    yy = re[0]
    dd = re[2]
    mm = re[1]
    fname = stn_code+dd+mm+yy+"."+yy+"N"
    with open(fname,'w') as f:
        for item in row:
            if index==0 and ind!=1:
                index=1
                f.write("%s"%item)
                pass
            elif index==1 and ind!=1:
                col=0
                for i in range(len(item)):                  
                    if col==2:
                        d = datetime.datetime.strptime(item[i], '%Y-%m-%d %H:%M:%S')
                        f.write("%s.0  "%datetime.datetime.strftime(d,'%y %m %d %H %M %S'))
                    elif not i==0:
                        if (col-1)%4==0 and not col==1:
                            f.write("%.12e  \n\t"%float(item[i]))
                        else:
                            if ind==2 and col==1:
                                f.write("  %s "%item[i])
                            elif col==1:
                                f.write("%s "%item[i])
                            else:
                                f.write("%s  "%item[i])
                    col+=1
            ind = ind + 1

df_final = pd.DataFrame()
df_final = df_final.append(pd.read_csv("csv_files/211251117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211261117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211271117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211281117.17N.csv"))
df_final = df_final.append(pd.read_csv("csv_files/211291117.17N.csv"))

df_expect = pd.read_csv("csv_files/211301117.17N.csv")

df_final['epoch_time']=pd.to_datetime(df_final['epoch_time'], format='%Y-%m-%d %H:%M:%S')
df_final = df_final.sort_values(by=['epoch_time'])
df_final.shape

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mapr = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return 0.0 if mapr!=mapr else mapr

df_predicted = pd.DataFrame()
acc_total = 0.0


def predict_each_satellite(prn):
    print(prn)
    df_temp = pd.DataFrame()
    avg_acc = 0.0

    df_prn = df_final.loc[df_final['prn'] == prn].reset_index() 
    df_expect2 = df_expect.loc[df_expect['prn'] == prn].reset_index()
    future = pd.DataFrame()
    future['ds'] = df_expect2['epoch_time']
    #print(future)

    df_m = pd.DataFrame()
    df_m['ds'] = df_prn['epoch_time']
    df_m['y'] = df_prn['epoch_time']
    model = Prophet()
    model.fit(df_m)
    #future = model.make_future_dataframe(periods=30, freq='H')
    df_temp['prn'] = prn
    df_temp['epoch_time'] = df_expect2['epoch_time']
    df_temp['prn'] = prn

    for column in list(df_prn.columns.values):
        #print('--------column--------- :'+ str(column))
        if column == 'prn' or column == 'epoch_time' or column=='index':
            continue
        
        if column == 'sv_clock_bias' or column == 'sv_clock_drift' or column == 'sv_clock_drift_rate' or column == 'mean_motion'  or  column == 'essentricity'  or column == 'sqrt_semi_major_axis'  or column == 'OMEGA' or column == 'inclination'  or column == 'omega' or column == 'OMEGA_dot' or column == 'inclination_rate' or column == 'codes' or column == 'gps_week' or column == 'l2_p_data_flag' or column == 'sv_accuracy' or column == 'sv_health' or column == 'tgd'  or column == 'fit_interval':
            df_m = pd.DataFrame()
            df_m['ds'] = df_prn['epoch_time']
            df_m['y'] = df_prn[column]

            model = Prophet()
            model.fit(df_m)

            forecast = model.predict(future)
            model.plot(forecast)
            #print('RMSE: %f' % (np.sqrt(np.mean((forecast.loc[:df2['y'].size-1, 'yhat']-df2['y'])**2))/np.abs(np.max(forecast.loc[:df2['y'].size-1, 'yhat']-df2['y']))))

            mse = mean_absolute_percentage_error(forecast['yhat'],df_expect2[column])
            #print("MSE: "+str(mse))
            avg_acc = avg_acc + mse
            df_temp[column] = forecast['yhat']
            del df_m
      
        if column == 'iode' or column == 'correction_radius_sine' or  column == 'correction_latitude_cosine' or column == 't_tx' or column == 'correction_latitude_sine' or  column == 'correction_inclination_cosine' or   column == 'correction_radius_cosine' or column == 'mean_anomaly' or  column == 'correction_inclination_sine':
            df_m = pd.DataFrame()
            df_m['ds'] = df_prn['epoch_time']
            df_m['y'] = df_prn[column]

            model = Prophet(changepoint_prior_scale=0.5)
            model.fit(df_m)

          
            forecast = model.predict(future)
            model.plot(forecast)
            #print('RMSE: %f' % (np.sqrt(np.mean((forecast.loc[:df2['y'].size-1, 'yhat']-df2['y'])**2))/np.abs(np.max(forecast.loc[:df2['y'].size-1, 'yhat']-df2['y']))))

            mse = mean_absolute_percentage_error(forecast['yhat'],df_expect2[column])
            #print("MSE: "+str(mse))
            avg_acc = avg_acc + mse
            df_temp[column] = forecast['yhat']
            del df_m
        
        if column == 'time_of_ephemeris' or column == 'iodc' :
            df_m = pd.DataFrame()
            df_m['ds'] = df_prn['epoch_time']
            df_m['y'] = df_prn[column]

            model = Prophet(changepoint_prior_scale=1.0)
            model.fit(df_m)

            forecast = model.predict(future)
            model.plot(forecast)

            #model.plot(forecast)
            #print('RMSE: %f' % (np.sqrt(np.mean((forecast.loc[:df2['y'].size-1, 'yhat']-df2['y'])**2))/np.abs(np.max(forecast.loc[:df2['y'].size-1, 'yhat']-df2['y']))))

            mse = mean_absolute_percentage_error(forecast['yhat'],df_expect2[column])
            #print("MSE: "+str(mse))
            avg_acc = avg_acc + mse
            df_temp[column] = forecast['yhat']
            del df_m
    

    #df_predicted = df_predicted.append(df_temp, ignore_index=True)
    print(df_temp.shape)

    print(" PRN : "+str(prn)+"  acc : "+str(avg_acc/30))

    return (df_temp,  avg_acc, prn)


if __name__ == '__main__':
    total_err = []
    prn_list = []
    a = datetime.datetime.now().replace(microsecond=0)
    input_prn = df_final['prn'].unique()
    pool = mp.Pool(processes = mp.cpu_count())
    for df_pred, err, prn in pool.map(predict_each_satellite, input_prn):
        df_predicted = df_predicted.append(df_pred, ignore_index=True)
        total_err.append(err)
        prn_list.append(prn)
    pool.close()
    pool.join()
    b = datetime.datetime.now().replace(microsecond=0)
    print(b-a)

    df_predicted.shape

    df_predicted

    df_predicted.reset_index()

    len(df_predicted['prn'].unique())

    df_predicted.to_csv(r'predicted_csv/predicted_5_1_1.csv')
    getdotn('predicted_csv/predicted_5_1_1.csv')


    err = []
    for e in total_err:
        err.append(e/30)

    plt.bar(prn_list, err)
    plt.xlabel('satellite number (prn)')
    plt.ylabel(' Error %')
    plt.show()

15
12
29
20


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbproph

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly season

INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.
/home/ask149/Env/gsc3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).

INF

...

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 16.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 15.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly season

(4, 31)
 PRN : 23  acc : 27.854663753195226


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 16.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 16.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 16.0.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly season

(4, 31)
 PRN : 7  acc : 25.0882549096468
0:17:18


FileNotFoundError: [Errno 2] No such file or directory: 'predicted_csv/predicted_5_1_1.csv'